In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch import Tensor
from data_generation import *
from torchvision import transforms
from PIL import Image, ImageDraw

In [ ]:
sequence, positions = generate_random_sequence()
display_sequence(sequence)
print(positions)

In [ ]:
# Hyperparameters
N = 1000  # Number of sequences
L = 10  # Length of each sequence
H, W = 16, 16  # Dimensions of the images
batch_size = 32
learning_rate = 0.001
num_epochs = 20


# Data Generation
class PixelDataset(Dataset):
    def __init__(self, num_sequences, sequence_length):
        self.data = []
        self.targets = []
        for _ in range(num_sequences):
            images, positions = generate_random_sequence()
            self.data.append(images)  # Store the whole sequence
            self.targets.append(positions)  # Store all positions

        self.transform = transforms.ToTensor()
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Transform and stack images to create a sequence
        sequence_of_images = [self.transform(image) for image in self.data[idx]]
        sequence_of_images = torch.stack(sequence_of_images)

        # Stack target positions
        target_positions = torch.FloatTensor(self.targets[idx])

        return sequence_of_images, target_positions


# Creating Dataset and DataLoader
dataset = PixelDataset(N, L)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-np.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[: x.size(0)]
        return self.dropout(x)

In [ ]:
class SimpleTransformer(nn.Module):
    def __init__(self):
        super(SimpleTransformer, self).__init__()
        self.encoder = nn.TransformerEncoderLayer(d_model=H * W, nhead=4)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder, num_layers=3)
        self.fc = nn.Linear(H * W, 2)  # Output is x, y position

    def forward(self, x):
        # Flatten the image and encode
        batch_size, sequence_length, _, _, _ = x.size()

        # 2D positional encoding
        # https://static.us.edusercontent.com/files/qjovJzQ2ERbedQv2ZeCZKAPS

        x = x.view(batch_size, sequence_length, -1)
        encoded = self.transformer_encoder(x)
        output = self.fc(encoded)  # Apply the linear layer to the entire sequence
        return output


def euclidean_distance_loss(output, target):
    epsilon = 1e-6  # A small constant to avoid sqrt(0)
    return torch.sqrt(torch.sum((output - target) ** 2, dim=-1) + epsilon).mean()


model = SimpleTransformer()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = euclidean_distance_loss

for epoch in range(num_epochs):
    for images, positions in dataloader:
        # Shift the positions by one to create the targets
        targets = torch.roll(positions, -1, dims=1)

        # Mask the last position in each sequence as it has no valid next position
        mask = torch.zeros_like(targets)
        mask[:, :-1, :] = 1  # Mask all but the last position

        # Forward pass
        predicted_positions = model(images)

        # Apply the mask
        masked_predicted = predicted_positions * mask
        masked_targets = targets * mask

        # Compute loss
        loss = criterion(masked_predicted, masked_targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

In [ ]:
def predict_sequence(model, sequence_of_images, device):
    """
    Predict the positions for a given sequence of images.

    :param model: Trained PyTorch model.
    :param sequence_of_images: A sequence of images as a PyTorch tensor.
    :param device: Device to perform computations on (e.g., 'cuda', 'cpu').
    :return: Predicted positions as a tensor.
    """
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        sequence_of_images = sequence_of_images.to(device)
        predicted_positions = model(sequence_of_images)
    return predicted_positions.cpu()


def create_images_from_positions(positions, image_size=(16, 16)):
    """
    Create a sequence of PIL images from a sequence of position tuples.
    Includes an empty image at the beginning of the sequence.

    :param positions: A sequence of (x, y) tuples as a PyTorch tensor.
    :param image_size: Size of the output images (width, height).
    :return: A list of PIL.Image.Image objects.
    """
    images = []
    positions = positions.squeeze()
    if positions.dim() == 1:
        positions = positions.unsqueeze(0)

    # Create and add an empty image for the first position
    empty_img = Image.new("1", image_size, 255)  # White image
    images.append(empty_img)

    for pos in positions:
        img = Image.new("1", image_size, 255)  # Create a white image
        draw = ImageDraw.Draw(img)
        x, y = pos.tolist()
        y = image_size[1] - 1 - y  # Adjust y-coordinate
        draw.point((x, y), fill=0)  # Draw the black pixel
        images.append(img)

    return images


def prepare_sequence_for_model(image_sequence):
    """
    Prepare a sequence of PIL images for input into the model.

    :param image_sequence: A list of PIL.Image.Image objects.
    :return: A PyTorch tensor representing the sequence.
    """
    transform = transforms.Compose(
        [
            transforms.Grayscale(),  # Convert to grayscale if not already
            transforms.ToTensor(),  # Convert PIL Image to PyTorch tensor
        ]
    )

    tensor_sequence = [transform(img) for img in image_sequence]
    tensor_sequence = torch.stack(tensor_sequence)  # Stack to create a single tensor
    tensor_sequence = tensor_sequence.unsqueeze(0)  # Add a batch dimension

    return tensor_sequence


def overlay_predicted_positions(s1, s2):
    """
    s1 black pixels stay black
    s2 black pixels become blue

    :param s1: A list of PIL.Image.Image objects representing the original sequence.
    :param s2: A list of PIL.Image.Image objects representing the predicted sequence.
    :return: A list of PIL.Image.Image objects with overlaid positions.
    """
    overlaid_images = []
    for img1, img2 in zip(s1, s2):
        img1 = img1.convert("RGB")  # Convert to RGB
        img2 = img2.convert("RGB")

        # Create a new image for the overlay
        new_img = Image.new("RGB", img1.size)
        pixels1 = img1.load()
        pixels2 = img2.load()

        for x in range(img1.width):
            for y in range(img1.height):
                if pixels2[x, y] == (0, 0, 0):  # Black pixel in s2
                    new_img.putpixel((x, y), (0, 0, 255))  # Make it blue
                else:
                    new_img.putpixel((x, y), pixels1[x, y])  # Use pixel from s1

        overlaid_images.append(new_img)

    return overlaid_images

In [ ]:
sequence, positions = generate_random_sequence()
sequence_of_images = prepare_sequence_for_model(sequence)
predicted_positions = predict_sequence(
    model, sequence_of_images, device="cpu"
)  # or 'cuda'
predicted_positions = predicted_positions[:, 1:, :]
predicted_images = create_images_from_positions(predicted_positions)
display_sequence(overlay_predicted_positions(sequence, predicted_images))